In [1]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
import random
import os

In [2]:
def alice_prepare_qubits(num_qubits):
    # Alice randomly prepares qubits in the |0⟩ or |1⟩ basis
    bits = [random.choice([0, 1]) for _ in range(num_qubits)]
    return bits

In [3]:
def encode_qubits(bits):
    # Encode qubits using the BBM92 protocol
    encoded_qubits = []
    for bit in bits:
        qc = QuantumCircuit(1, 1)
        if bit == 1:
            qc.x(0)  # Apply X gate for |1⟩ basis
        qc.h(0)      # Apply Hadamard gate
        encoded_qubits.append(qc)
    return encoded_qubits

In [4]:
def bob_measure_qubits(encoded_qubits):
    # Bob measures qubits randomly in the |0⟩ or |1⟩ basis
    backend = Aer.get_backend('qasm_simulator')
    measurements = []
    for qc in encoded_qubits:
        qc.measure(0, 0)
        t_qc = transpile(qc, backend)
        qobj = assemble(t_qc)
        result = backend.run(qobj).result()
        counts = result.get_counts()
        measured_bit = int(max(counts, key=counts.get))
        measurements.append(measured_bit)
    return measurements

In [5]:
def alice_select_bases(num_qubits):
    # Alice randomly selects measurement bases
    bases = [random.choice([0, 1]) for _ in range(num_qubits)]
    return bases

In [6]:
def compare_bases(alice_bases, bob_bases):
    # Compare Alice's and Bob's selected bases
    matching_bases = [alice_bases[i] == bob_bases[i] for i in range(len(alice_bases))]
    return matching_bases

In [7]:
def key_from_bits(bits, matching_bases):
    # Extract a key from the matching bits
    key = [bits[i] for i in range(len(bits)) if matching_bases[i]]
    return key

In [8]:
def encrypt_decrypt_file(input_file, output_file):
    # Read the content of the input file
    with open(input_file, 'r') as file:
        original_text = file.read()

    num_qubits = len(original_text) * 8  # Assuming each character is 8 bits

    # Convert text to a list of bits
    bits = [int(bit) for char in original_text for bit in bin(ord(char))[2:].zfill(8)]

    # Alice prepares qubits
    alice_bits = alice_prepare_qubits(num_qubits)

    # Alice encodes qubits using BBM92
    encoded_qubits = encode_qubits(alice_bits)

    # Bob measures qubits
    bob_bits = bob_measure_qubits(encoded_qubits)

    # Alice selects measurement bases
    alice_bases = alice_select_bases(num_qubits)

    # Compare Alice's and Bob's bases
    matching_bases = compare_bases(alice_bases, bob_bits)

    # Extract key from matching bits
    key = key_from_bits(alice_bits, matching_bases)

    # XOR the key with the original bits to encrypt
    encrypted_bits = [bit ^ key_bit for bit, key_bit in zip(bits, key)]

    # Write the encrypted bits to the output file
    with open(output_file, 'wb') as file:
        file.write(bytes(encrypted_bits))

    # XOR the encrypted bits with the key to decrypt
    decrypted_bits = [bit ^ key_bit for bit, key_bit in zip(encrypted_bits, key)]

    # Convert bits back to text
    decrypted_text = ''.join([chr(int(''.join(map(str, decrypted_bits[i:i+8])), 2)) for i in range(0, len(decrypted_bits), 8)])

    return decrypted_text

In [9]:
def main():
    input_file = "input.txt"
    encrypted_file = "encrypted_file.bin"
    decrypted_text = encrypt_decrypt_file(input_file, encrypted_file)

    print("Original Text:", decrypted_text)

if __name__ == "__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'input.txt'